### Symantecのattack signatureページを読み込んで見る

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup 
import re

html = urlopen('https://www.symantec.com/security_response/attacksignatures/')
bs = BeautifulSoup(html, 'html.parser')

In [2]:
#URLのリストを作成する
def makeURL(link):
    return 'https://www.symantec.com'+link

links = bs.find('div', {'class': 'bckPadLG bckSolidWht unit'}).findAll('a',href=re.compile('^(/security_response/)'))
URLs=[makeURL(l.attrs['href']) for l in links]
len(URLs)

6459

In [ ]:
def get_attack_details(link):
    '''
    return title and bs object
    '''
    html = urlopen(link)
    bs= BeautifulSoup(html, 'html.parser') 
    return {'name':bs.h1,
            'contents':bs.find('div', {'class':'bckPadLG bckSolidWht unit'})}

# get_attack_details(URLs[0])
contents=[get_attack_details(l) for l in URLs]

しかしここで、問題が発生した。exploit dateが見当たらないことである。論文中では活用されていたのになぜ...

### Symantecの[security-center](https://www.symantec.com/security-center/a-z/E)からURLを取得してスクレーピングする
URLの一覧はhttps://www.symantec.com/security-center/srlistingRssFeedXml/E.rss.feed から取得した方が楽そう

In [8]:
from urllib.request import urlopen
from bs4 import BeautifulSoup 
import re
import lxml

html = urlopen('https://www.symantec.com/security-center/srlistingRssFeedXml/E.rss.feed')
bs = BeautifulSoup(html, 'xml')

In [25]:
links=list(map(lambda x:x.get_text(),bs.find_all('link')))

In [28]:
html = urlopen(links[100])
bs = BeautifulSoup(html, 'html.parser')

In [90]:
title = bs.h1.get_text()
cve_ie=title[4:]
exp_date = bs.find('section', {'class':"content-band"}).find(text=re.compile(r'.* \d{2}, \d{4}'))

In [92]:
title,cve_ie,exp_date

('Exp.CVE-2015-0338', 'CVE-2015-0338', 'August 03, 2015')

他に収集すべき情報は？(教師データなので、CVE-IDと評価時に必要な日付情報だけあれば良い気もするが)

In [ ]:
bs.

In [30]:
links[100]

'http://www.symantec.com/security_response/writeup.jsp?docid=2015-080322-5112-99&om_rssid=sr-'